<a href="https://colab.research.google.com/github/wjleece/anthropic-prompts/blob/main/Anthropic_Customer_Service_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%pip install anthropic

In [35]:
import anthropic
import re
from google.colab import userdata

API_KEY = userdata.get('ANTHROPIC_API_KEY')
MODEL_NAME = "claude-3-7-sonnet-latest"
client = anthropic.Anthropic(api_key=API_KEY)

In [36]:
system_prompt = "You are a helpful assistant."

In [37]:
tools = []

In [38]:
def get_completion(prompt: str):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        tools=tools,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [39]:
prompt="Yo Claude, wuzzup?"

print(get_completion(prompt))

Hey there! Not much on my end, just hanging out in the digital realm, ready to help with whatever you need. What's going on with you today? Anything I can help you with or chat about?


In [56]:
tools = [
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
     {
        "name": "add_customer",
        "description": "Adds a new customer to the database. Includes customer ID, name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer, which you automatically will generate any time add_customer is called."
                },
                "name": {
                    "type": "string",
                    "description": "The name of the customer."
                },
                "email": {
                    "type": "string",
                    "description": "The email address of the customer."
                },
                "phone": {
                    "type": "string",
                    "description": "The phone number of the customer."
                }
            },
            "required": ["customer_id", "name", "email"],
            "optional": ["phone"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                },
                "product": {
                    "type": "string",
                    "description": "The name of the product in the order."
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the product in the order."
                },
                "price": {
                    "type": "number",
                    "description": "The price of each product in the order."
                },
                "status": {
                    "type": "string",
                    "description": "The current status of the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "add_order",
        "description": "Creates an order. Includes order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order, which you automatically will generate any time add_order is called."
                },
                "product": {
                    "type": "string",
                    "description": "The name of the product in the order."
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the product in the order."
                },
                "price": {
                    "type": "number",
                    "description": "The price of each product in the order."
                },
                "status": {
                    "type": "string",
                    "description": "The current status of the order."
                }
            },
            "required": ["order_id", "product", "quantity", "price", "status" ]
        }
    },
    {
        "name": "cancel_order",
        "description": """Cancels an order based on the provided order ID. Sets the order status to "Canceled" and returns a confirmation message if the cancellation is successful.""",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

In [70]:
# Global customer storage
customers = {
    "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
    "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
}

# Global order storage
orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }

def get_customer_info(customer_id):
    return customers.get(customer_id, "Customer not found")

def add_customer(name, email, phone):
    new_id = f"C{len(customers) + 1}"  # Simple ID generator like C3, C4, ...
    customers[new_id] = {"name": name, "email": email, "phone": phone}
    return new_id, customers[new_id]

def get_order_details(order_id):
    # Simulated order data
    return orders.get(order_id, "Order not found")

def add_order(product, quantity, price, status):
    new_id = f"O{len(orders) + 1}"  # Simple ID generator like O
    orders[new_id] = {"id": new_id, "product": product, "quantity": quantity, "price": price, "status": status}
    return new_id, orders[new_id]

def cancel_order(order_id):
    # Check if the order exists
    if order_id in orders:
        # Keep all existing order data and just update the status
        orders[order_id]["status"] = "Canceled"
        return order_id, orders[order_id]
    else:
        return "Order not found"


In [71]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "add_customer":
        return add_customer(tool_input["name"], tool_input["email"], tool_input["phone"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "add_order":
        return add_order(tool_input["product"], tool_input["quantity"], tool_input["price"], tool_input["status"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

In [72]:
import json

def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response

In [75]:
chatbot_interaction("Can you tell me the email address for customer C1?")


User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you find the email address for customer C1. Let me retrieve this information for you.", type='text'), ToolUseBlock(id='toolu_014VcmhaMFjiYNcSsXKe1jD7', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Tool Used: get_customer_info
Tool Input:
{
  "customer_id": "C1"
}

Tool Result:
{
  "name": "John Doe",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="Here's the email address for customer C1 (John Doe): john@example.com\n\nWould you like me to provide any other information about this customer?", type='text')]

Final Response: Here's the email address for customer C1 (John Doe): john@example.com

Would you like me to provide any other information about this customer?


"Here's the email address for customer C1 (John Doe): john@example.com\n\nWould you like me to provide any other information about this customer?"

In [76]:
chatbot_interaction("What is the status of order O2?")


User Message: What is the status of order O2?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll check the status of order O2 for you. Let me retrieve the order details.", type='text'), ToolUseBlock(id='toolu_0166AoTJSkZ4YxbWhHARn3ye', input={'order_id': 'O2'}, name='get_order_details', type='tool_use')]

Tool Used: get_order_details
Tool Input:
{
  "order_id": "O2"
}

Tool Result:
{
  "id": "O2",
  "product": "Gadget B",
  "quantity": 1,
  "price": 49.99,
  "status": "Processing"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='I\'ve checked the status of order O2 for you. The order is currently in "Processing" status. This order contains 1 unit of "Gadget B" at a price of $49.99.', type='text')]

Final Response: I've checked the status of order O2 for you. The order is currently in "Processing" status. This order contains 1 unit of "Gadget B" at a price of $49.99.


'I\'ve checked the status of order O2 for you. The order is currently in "Processing" status. This order contains 1 unit of "Gadget B" at a price of $49.99.'

In [77]:
chatbot_interaction("Please cancel order O1 for me.")


User Message: Please cancel order O1 for me.

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you cancel order O1. Let me process that for you right away.", type='text'), ToolUseBlock(id='toolu_018CFHCpGrF92xsj1m8SWYXE', input={'order_id': 'O1'}, name='cancel_order', type='tool_use')]

Tool Used: cancel_order
Tool Input:
{
  "order_id": "O1"
}

Tool Result:
[
  "O1",
  {
    "id": "O1",
    "product": "Widget A",
    "quantity": 2,
    "price": 19.99,
    "status": "Canceled"
  }
]

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='Your order O1 for "Widget A" has been successfully canceled. The system has updated the order status to "Canceled". \n\nIs there anything else you need help with regarding your orders?', type='text')]

Final Response: Your order O1 for "Widget A" has been successfully canceled. The system has updated the order status to "Canceled". 

Is there anything else you need help with regarding your 

'Your order O1 for "Widget A" has been successfully canceled. The system has updated the order status to "Canceled". \n\nIs there anything else you need help with regarding your orders?'

In [78]:
print(get_order_details("O1"))

{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Canceled'}


In [79]:
chatbot_interaction("Add a new customer: Bill Leece, bill.leece@example.com, 111-222-3333")


User Message: Add a new customer: Bill Leece, bill.leece@example.com, 111-222-3333

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you add Bill Leece as a new customer with the provided information. I'll use the add_customer function to create this new record.", type='text'), ToolUseBlock(id='toolu_01WrWmcXEdyRn5CK8a2hLvDd', input={'customer_id': 'CUST-001', 'name': 'Bill Leece', 'email': 'bill.leece@example.com', 'phone': '111-222-3333'}, name='add_customer', type='tool_use')]

Tool Used: add_customer
Tool Input:
{
  "customer_id": "CUST-001",
  "name": "Bill Leece",
  "email": "bill.leece@example.com",
  "phone": "111-222-3333"
}

Tool Result:
[
  "C3",
  {
    "name": "Bill Leece",
    "email": "bill.leece@example.com",
    "phone": "111-222-3333"
  }
]

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="Great! I've successfully added Bill Leece to the customer database with the following details:\n- Customer ID: C

"Great! I've successfully added Bill Leece to the customer database with the following details:\n- Customer ID: C3\n- Name: Bill Leece\n- Email: bill.leece@example.com\n- Phone: 111-222-3333\n\nThe customer has been registered in our system and is now ready for placing orders."

In [80]:
customers

{'C1': {'name': 'John Doe',
  'email': 'john@example.com',
  'phone': '123-456-7890'},
 'C2': {'name': 'Jane Smith',
  'email': 'jane@example.com',
  'phone': '987-654-3210'},
 'C3': {'name': 'Bill Leece',
  'email': 'bill.leece@example.com',
  'phone': '111-222-3333'}}

In [82]:
chatbot_interaction("Create a new order for: Perplexinator, 12, $59.99, Shipped")


User Message: Create a new order for: Perplexinator, 12, $59.99, Shipped

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you create a new order for the Perplexinator product. I have most of the details needed, but I'll need to generate an order ID automatically when creating the order. Let me process this for you:", type='text'), ToolUseBlock(id='toolu_014gRezJ2CeFeRf9BXyJpN6w', input={'order_id': 'ORD-2023-7845', 'product': 'Perplexinator', 'quantity': 12, 'price': 59.99, 'status': 'Shipped'}, name='add_order', type='tool_use')]

Tool Used: add_order
Tool Input:
{
  "order_id": "ORD-2023-7845",
  "product": "Perplexinator",
  "quantity": 12,
  "price": 59.99,
  "status": "Shipped"
}

Tool Result:
[
  "O3",
  {
    "id": "O3",
    "product": "Perplexinator",
    "quantity": 12,
    "price": 59.99,
    "status": "Shipped"
  }
]

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="Great! I've successfully created a new o

"Great! I've successfully created a new order with the following details:\n\n- Order ID: O3\n- Product: Perplexinator\n- Quantity: 12\n- Price: $59.99\n- Status: Shipped\n\nThe order has been entered into the system and is marked as shipped. Is there anything else you need assistance with regarding this order?"

In [83]:
orders

{'O1': {'id': 'O1',
  'product': 'Widget A',
  'quantity': 2,
  'price': 19.99,
  'status': 'Canceled'},
 'O2': {'id': 'O2',
  'product': 'Gadget B',
  'quantity': 1,
  'price': 49.99,
  'status': 'Processing'},
 'O3': {'id': 'O3',
  'product': 'Perplexinator',
  'quantity': 12,
  'price': 59.99,
  'status': 'Shipped'}}